In [43]:
import pandas as pd
from sklearn import tree
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

In [6]:
cleanedData = pd.read_csv('./Fully-Cleaned-Data.csv')

In [7]:
categoricalColumns = cleanedData.select_dtypes(include=['object']).columns.tolist()
numericalData = cleanedData.drop(columns=categoricalColumns)
oneHotData = pd.get_dummies(cleanedData[categoricalColumns])
print(numericalData.shape)
print(numericalData.columns.values)
print(oneHotData.shape)
print(oneHotData.columns.values)

(2047, 19)
['AgeatFV' 'DiseasedurationatFV' 'EDSS_FV' 'PYRAMIDAL_FUNCTION'
 'CEREBELLAR_FUNCTION' 'BRAINSTEM_FUNCTION' 'SENSORY_FUNCTION'
 'BOWEL_BLADDER_FUNCTION' 'VISUAL_FUNCTION' 'MENTAL_FUNCTION'
 'TotalnumberofrelapsesbeforeFV' 'Numberofrelapsesinthe3yearsbeforeFV'
 'Numberofrelapsesinthe1yearbeforeFV' 'timeSinceLastAttack'
 'Future Relapse Binary' 'TreatmentBeforeFV' 'RelapseInYearBeforeFVBinary'
 'RelapseInThe3YearsBeforeFVBinary' 'Treatment with Injectable Med']
(2047, 29)
['SEX_F' 'SEX_M' 'RACE_DESC_American Indian or Alaska Native'
 'RACE_DESC_Asian' 'RACE_DESC_Black or African American'
 'RACE_DESC_East Asian and South-East' 'RACE_DESC_More than one race'
 'RACE_DESC_Native Hawaiian or Other Pacific Islander'
 'RACE_DESC_South Asian' 'RACE_DESC_Unknown or not reported'
 'RACE_DESC_White' 'ETHNICITY_DESC_Hispanic or latino'
 'ETHNICITY_DESC_Not hispanic or latino' 'ETHNICITY_DESC_Unknown'
 'FAMILY_MS_N' 'FAMILY_MS_U' 'FAMILY_MS_Y' 'SMOKING_EVER_N'
 'SMOKING_EVER_Y' 'DISEASE_C

In [8]:
cleanedDataOneHotEncoded = pd.concat([numericalData, oneHotData], axis=1)
print(cleanedDataOneHotEncoded.shape)

(2047, 48)


In [45]:
import warnings
warnings.filterwarnings('ignore')

treeClassifier = tree.DecisionTreeClassifier(max_depth=5, class_weight='balanced')
indicator = cleanedDataOneHotEncoded['Future Relapse Binary']
X = cleanedDataOneHotEncoded.drop(['Future Relapse Binary'], axis=1)
treeModel = treeClassifier.fit(X, indicator)

predictions = pd.DataFrame(treeModel.predict(X))
print('True Class Breakdown')
print(indicator.value_counts())
print('Predicted Class Breakdown')
print(predictions.value_counts())



True Class Breakdown
Future Relapse Binary
0    1315
1     732
Name: count, dtype: int64
Predicted Class Breakdown
0
1    1298
0     749
Name: count, dtype: int64


In [46]:
accScore = metrics.accuracy_score(indicator, predictions)
precScore = metrics.precision_score(indicator, predictions)
recScore = metrics.recall_score(indicator, predictions)
yeet = metrics.f1_score(indicator, predictions)
print(accScore)
print(precScore)
print(recScore)
print(yeet)

cm = confusion_matrix(indicator, predictions)
print(cm)

cr = classification_report(indicator, predictions)
print(cr)

0.6072300928187592
0.47226502311248075
0.837431693989071
0.6039408866995074
[[630 685]
 [119 613]]
              precision    recall  f1-score   support

           0       0.84      0.48      0.61      1315
           1       0.47      0.84      0.60       732

    accuracy                           0.61      2047
   macro avg       0.66      0.66      0.61      2047
weighted avg       0.71      0.61      0.61      2047

